# 🔧 Digital Sahayak - Hyperparameter Management

This notebook demonstrates how to manage hyperparameters using **separate configuration files** instead of hardcoding values.

## Benefits:
- ✅ Easy to modify without changing code
- ✅ Better experiment tracking
- ✅ Shareable configurations
- ✅ Version control friendly
- ✅ Type-safe with validation

## 1. Import Required Libraries

In [ ]:
import os
import sys
import json
import yaml
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import Dict, Any, Optional, List

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent))

print("✅ Libraries imported successfully!")

## 2. Define Hyperparameter Dataclasses

Type-safe configuration with proper defaults:

In [ ]:
@dataclass
class TrainingConfig:
    """Training hyperparameters"""
    epochs: int = 10
    batch_size: int = 32
    learning_rate: float = 2e-5
    weight_decay: float = 0.01
    warmup_steps: int = 500
    max_grad_norm: float = 1.0
    
@dataclass
class OptimizerConfig:
    """Optimizer settings"""
    type: str = "adamw"
    betas: List[float] = field(default_factory=lambda: [0.9, 0.999])
    eps: float = 1e-8

@dataclass
class ModelConfig:
    """Model architecture"""
    type: str = "transformer"
    pretrained_model: str = "ai4bharat/indic-bert"
    hidden_size: int = 768
    num_labels: int = 15
    dropout: float = 0.3

@dataclass
class IntentClassifierConfig:
    """Complete config for Intent Classifier"""
    model: ModelConfig = field(default_factory=ModelConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    optimizer: OptimizerConfig = field(default_factory=OptimizerConfig)
    
    def to_dict(self) -> Dict:
        return asdict(self)

# Create default config
default_config = IntentClassifierConfig()
print("📋 Default Configuration:")
print(json.dumps(default_config.to_dict(), indent=2))

## 3. Load Configuration from YAML File

In [ ]:
# Load from our config file
config_path = Path.cwd().parent / "ai" / "config" / "hyperparameters.yaml"

with open(config_path, 'r', encoding='utf-8') as f:
    full_config = yaml.safe_load(f)

print(f"✅ Loaded config from: {config_path}")
print(f"\n📋 Available models: {list(full_config.keys())}")

# Get intent classifier config
intent_config = full_config.get('intent_classifier', {})
print(f"\n🧠 Intent Classifier Hyperparameters:")
print(f"  Learning Rate: {intent_config['training']['learning_rate']}")
print(f"  Batch Size: {intent_config['training']['batch_size']}")
print(f"  Epochs: {intent_config['training']['epochs']}")
print(f"  Optimizer: {intent_config['optimizer']['type']}")

## 4. Validate Hyperparameters

In [ ]:
def validate_hyperparams(config: Dict) -> List[str]:
    """Validate hyperparameters and return list of errors"""
    errors = []
    
    # Training validation
    training = config.get('training', {})
    
    if training.get('learning_rate', 0) <= 0:
        errors.append("learning_rate must be positive")
    if training.get('learning_rate', 0) > 0.1:
        errors.append("learning_rate > 0.1 is too high")
        
    if training.get('batch_size', 0) <= 0:
        errors.append("batch_size must be positive")
    if training.get('batch_size', 0) > 512:
        errors.append("batch_size > 512 may cause OOM")
        
    if training.get('epochs', 0) <= 0:
        errors.append("epochs must be positive")
    if training.get('epochs', 0) > 100:
        errors.append("epochs > 100 may overfit")
    
    # Model validation
    model = config.get('model', {})
    if model.get('dropout', 0) < 0 or model.get('dropout', 0) > 1:
        errors.append("dropout must be between 0 and 1")
    
    return errors

# Validate our config
errors = validate_hyperparams(intent_config)
if errors:
    print("❌ Validation Errors:")
    for e in errors:
        print(f"  - {e}")
else:
    print("✅ All hyperparameters are valid!")

## 5. Use ConfigManager from Our Module

In [ ]:
# Import our config module
from ai.config import config, get_hyperparams

# Get hyperparameters with dot notation
lr = config.get("intent_classifier.training.learning_rate")
batch_size = config.get("intent_classifier.training.batch_size")
epochs = config.get("intent_classifier.training.epochs")

print("📊 Using ConfigManager:")
print(f"  Learning Rate: {lr}")
print(f"  Batch Size: {batch_size}")
print(f"  Epochs: {epochs}")

# Get complete hyperparams as flat dict
hyperparams = get_hyperparams("intent_classifier")
print(f"\n📋 Flattened Hyperparams: {list(hyperparams.keys())}")

## 6. Example: Training Loop with Config

In [ ]:
def train_model(model_name: str = "intent_classifier"):
    """
    Example training function that uses config file for hyperparameters.
    
    Instead of hardcoding:
        lr = 0.00002  # ❌ Bad practice
        
    We use:
        lr = config.get("intent_classifier.training.learning_rate")  # ✅ Good practice
    """
    # Load hyperparameters from config
    hp = get_hyperparams(model_name)
    
    print(f"🚀 Training {model_name} with:")
    print(f"  Epochs: {hp['epochs']}")
    print(f"  Batch Size: {hp['batch_size']}")
    print(f"  Learning Rate: {hp['learning_rate']}")
    print(f"  Optimizer: {hp['optimizer_type']}")
    
    # Simulated training loop
    for epoch in range(1, min(hp['epochs'] + 1, 4)):  # Only 3 epochs for demo
        train_loss = 1.0 / epoch  # Simulated
        print(f"  Epoch {epoch}: loss = {train_loss:.4f}")
    
    print("✅ Training complete!")
    return {"final_loss": train_loss}

# Run training
result = train_model()

## 7. View All Model Configurations

In [ ]:
# Compare all model configs
models = ['intent_classifier', 'field_classifier', 'job_recommender', 'content_rewriter']

print("📊 Model Hyperparameter Comparison:\n")
print(f"{'Model':<20} {'Epochs':<10} {'Batch Size':<12} {'Learning Rate':<15}")
print("-" * 60)

for model in models:
    cfg = config.get_training_config(model)
    if cfg:
        print(f"{model:<20} {cfg.get('epochs', '-'):<10} {cfg.get('batch_size', '-'):<12} {cfg.get('learning_rate', '-'):<15}")